 將vtreport副檔名改為txt

In [ ]:
# ## add extension .txt
# for filename in os.listdir('data/virus_total/'):
#     base_file, ext = os.path.splitext(filename)
#     if ext != ".txt":
#         os.rename('data/virus_total/'+filename, 'data/virus_total/'+filename + ".txt")

In [ ]:
def pre_search(in_directory,in_hooklog_directory,in_first_seen=True):
# MIKE: 20170822, hack for TXT (VT report) or hooklog
#     run_directory = in_hooklog_directory if in_hooklog_directory != None else in_directory

    # iter the directory
    file_list = next(os.walk(in_hooklog_directory))[2]
#     print(file_list)
    hash_set = set(t.split('.')[0].split("_")[0] for t in file_list)
    ext = file_list[0].split('.')[-1].lower() 

    print("%d files" % len(file_list))
    print("%d hashes" % len(hash_set))
#     print(ext)
#     print("save to", out_csvfile)
#     print("save to", out_wn_csvfile)

    # find all anti-virus engines and corresponding detection strings

    av_set = set() # set of all anti-virus engines
    csv_dict = dict()
    unknown = []

    for h in hash_set:
        # open txt file and load it as json
        try:
            with open(os.path.join(in_directory, h + '.txt')) as txt_file:    
                json_report = json.load(txt_file)
#                 print(json_report)
        except:
            unknown.append(os.path.join(in_directory, h ).split('/')[-1])

        # create a dictionary _dict = {engine: "detection_name"}
        try:
            _dict = dict()
            for engine in json_report['scans'].keys(): 
                scan_result = json_report['scans'].get(engine)
                if scan_result.get("detected") == True:
                    result = scan_result.get("result").encode('ascii', 'ignore')
                    result = result.decode("ascii").replace(',', '') # special replacement for csv
                    av_set.add(engine)

                    _dict[engine] = result

            # if you don't need first_seen, set in_first_seen as Fasle
            if in_first_seen:
                _dict["first_seen"] = json_report['first_seen']

            # attach this dictioary to csv_dict = {hash: _dict}
            csv_dict[h] = _dict
        except Exception as e:
            print('!!!!!!!!!!!!!!!!!',str(e),'!!!!!!!!!!!!!!!!!')
            pass

    print("沒有vtreport的數量 / main全部的hash數目 :",len(unknown), "/", len(hash_set))
    # unknown

    df = pd.DataFrame(csv_dict).T
    return df
# You can print the df here
# df.head()
#df['AVG']

In [ ]:
def split_T_V(df,in_hooklog_directory,out_train,out_valid,split_rate=0.9):    
    sorts = df.filter(['first_seen'])
    sorts = sorts.sort_values(['first_seen'],ascending=[1])
    sorts = sorts.reset_index()
    sorts.columns=['hls','time']
    sort = sorts.drop(columns='time')
#     sort

    split_T = int(len(sorts)*split_rate)
    sort_T , sort_V = sort[:split_T], sort[split_T:] 
    sort_V.reset_index(inplace=True)

    if not os.path.isdir(out_train):
        os.makedirs(out_train)
    for hls in sort_T['hls']:
        pattern_ = in_hooklog_directory + str(hls) + "*"
#         print(pattern_)
        whole_name = glob.glob(pattern_)[0]
        shutil.copy(whole_name, os.path.join(out_train, whole_name.split('/')[-1]))
    print(os.path.join(out_train))
    if not os.path.isdir(out_valid):
        os.makedirs(out_valid)
    for hls in sort_V['hls']:
        pattern_ = in_hooklog_directory + str(hls) + "*"
        whole_name = glob.glob(pattern_)[0]
        shutil.copy(whole_name, os.path.join(out_valid,whole_name.split('/')[-1]))
    print(os.path.join(out_valid))
#     return

In [ ]:
## for android

def main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate):
    
    df = pre_search(in_directory,in_hooklog_directory)
    try:
        split_T_V(df,in_hooklog_directory,out_train,out_valid,split_rate)
    except KeyError as e:
        print("!!!!!!!!","KeyError: ",str(e),'!!!!!!!!!!!')
        pass
    


    return

if __name__ == '__main__':
    import os
    import json
    import pandas as pd
    from numpy import nan as NaN
    import shutil
    import glob
#     in_tag = "autoit/main/"

    #PARAMETERS
    in_directory = "./data/VTReport/" # virus total private key report(包含firstseen)，底下是全部的 .txt
    in_hooklog_directorys = "./data/tracelogs_analysis_temu20/" # hooklog的根目錄，下面有"家族family name"名稱，在下面有"代別generation"名稱
    split_rate = 0.9 #多少%要當訓練資料
    out_trains = 'data/train/' #輸出訓練hooklogs
    out_valids = 'data/test/'  #輸出測試hooklogs

    
    for family in next(os.walk(in_hooklog_directorys))[1]:
        print("==========",str(family)+"/","==========")
        in_hooklog_directory = in_hooklog_directorys+family+"/"
        out_train = out_trains+family+"/"
        out_valid = out_valids + family+"/"
        main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate=0.9)

#     in_first_seen = True
